#引入所需套件

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

#證交所api連線

In [ ]:
# 將日期從民國年轉換為西元年
def convert_roc_to_ad(date_str):
    try:
        parts = date_str.split('/')
        roc_year = int(parts[0])
        ad_year = roc_year + 1911  # 將民國年轉換為西元年
        return f'{ad_year}/{parts[1]}/{parts[2]}'
    except Exception as e:
        print(f"日期轉換錯誤: {date_str}, {e}")
        return date_str

# 從台灣證交所爬取當日股票資料
def get_tse_stock_price(stock_id):
    try:
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={datetime.now().strftime("%Y%m%d")}&stockNo={stock_id}'
        response = requests.get(url)
        response.raise_for_status()  # 檢查請求是否成功
        data = response.json()

        # 確認回傳數據中是否包含有效數據
        if 'data' not in data or not data['data']:
            print(f"無法獲取 {stock_id} 的數據。")
            return pd.DataFrame()

        # 將數據轉換為 DataFrame
        title=data['title']
        df = pd.DataFrame(data['data'], columns=data['fields'])
        df['日期'] = df['日期'].apply(convert_roc_to_ad)
        df['日期'] = pd.to_datetime(df['日期'], format='%Y/%m/%d', errors='coerce')  # 加入錯誤處理

        return df,title

    except requests.exceptions.RequestException as e:
        print(f"網路請求錯誤: {e}")
        return pd.DataFrame()


stock_id = '2330'
tse_df,title = get_tse_stock_price(stock_id)
print(f'{title}\n')
print(tse_df.to_markdown())

113年09月 2330 台積電           各日成交資訊

|    | 日期                | 成交股數   | 成交金額       |   開盤價 |   最高價 |   最低價 |   收盤價 | 漲跌價差   | 成交筆數   |
|---:|:--------------------|:-----------|:---------------|---------:|---------:|---------:|---------:|:-----------|:-----------|
|  0 | 2024-09-02 00:00:00 | 19,272,593 | 18,270,058,260 |      950 |      955 |      943 |      948 | +4.00      | 31,642     |
|  1 | 2024-09-03 00:00:00 | 23,205,623 | 21,908,471,541 |      948 |      952 |      939 |      940 | -8.00      | 47,139     |
|  2 | 2024-09-04 00:00:00 | 93,169,835 | 83,424,133,824 |      894 |      905 |      888 |      889 | -51.00     | 393,210    |
|  3 | 2024-09-05 00:00:00 | 34,147,890 | 30,998,595,394 |      907 |      915 |      900 |      902 | +13.00     | 63,902     |
|  4 | 2024-09-06 00:00:00 | 28,248,063 | 25,786,016,936 |      909 |      918 |      903 |      918 | +16.00     | 38,415     |
|  5 | 2024-09-09 00:00:00 | 38,448,946 | 34,456,838,126 |      892 |      900 |      891 | 

#Yahoo 奇摩

In [59]:
# 爬取 Yahoo 奇摩股價資料
def get_yahoo_stock_price(stock_id):
    url = f"https://tw.stock.yahoo.com/quote/{stock_id}.TW"

    # 發送請求，獲取 HTML 內容
    response = requests.get(url)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"無法獲取 {stock_id} 的數據。")
        return pd.DataFrame()

    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(response.text, "html.parser")
    # 查找當前股價的元素（需要根據實際的 Yahoo 股價頁面 HTML 結構調整）
    price_tag = soup.find('span', {'class': 'Fz(32px)'})
    titleNum_tag = soup.find('span', {'class': 'Fz(24px)'})
    titleTxt_tag = soup.find('h1', {'class': 'Fz(24px)'})
    if price_tag and titleNum_tag and titleTxt_tag:
        current_price = price_tag.text
        print(f"個股: {titleTxt_tag.text}({titleNum_tag.text})")
        print(f"目前股價: {current_price}")
    else:
        print(f"無法在頁面中找到 {stock_id} 的股價資料。")
        return pd.DataFrame()

    # 查找表格中的數據
    tables = soup.find_all('li', {'class': 'price-detail-item'})

    # 檢查是否有找到表格
    if not tables:
        print(f"無法在頁面中找到 {stock_id} 的股價資料。")
        return pd.DataFrame()

    # 儲存數據的字典
    stock_data = {}

    # 遍歷每個表格來提取相關數據
    for table in tables:
        spans = table.find_all('span')
        if len(spans) == 2:
            key = spans[0].text.strip()  # 第一個 span 作為鍵
            value = spans[1].text.strip()  # 第二個 span 作為值
            stock_data[key] = value

    # 打印提取出的數據
    for key, value in stock_data.items():
        print(f"{key}: {value}")

    # 將數據轉換為 DataFrame
    df = pd.DataFrame(list(stock_data.items()), columns=['項目', '數值'])

    return df

stock_id = '2330'
yahoo_df = get_yahoo_stock_price(stock_id)


個股: 台積電(2330)
目前股價: 987
成交: 987
開盤: 976
最高: 987
最低: 971
均價: 979
成交金額(億): 261.37
昨收: 977
總量: 26,678
昨量: 14,456
振幅: 1.64%
